# Lecture 10 - Code Implementation for the Naive Bayes Classifier & Its Discriminant Functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')
import scipy.stats as stats
import pandas as pd

In [ ]:
def generateData(mean1, mean2, cov1, cov2, N1, N2):
    # We are generating data from two Gaussians to represent two classes
    # In practice, we would not do this - we would just have data from the problem we are trying to understand
    data_C1 = stats.multivariate_normal(mean1, cov1).rvs(size=N1)
    data_C2 = stats.multivariate_normal(mean2, cov2).rvs(size=N2)
        
    # Entire Training Dataset
    data = np.concatenate((data_C1, data_C2))
    labels = np.concatenate((np.ones(N1),2*np.ones(N2)))
    
    if len(mean1)>1:
        plt.scatter(data[labels==1,0], data[labels==1,1], c='b', alpha=0.5, edgecolors='k')
        plt.scatter(data[labels==2,0], data[labels==2,1], c='r', alpha=0.5, edgecolors='k')
        plt.xlabel('Feature 1'); plt.ylabel('Feature 2');
    else:
        plt.scatter(data[labels==1], np.ones(N1), c='b', alpha=0.5, edgecolors='k')
        plt.scatter(data[labels==2], np.ones(N2), c='r', alpha=0.5, edgecolors='k')
        plt.xlabel('Feature 1');
        
    return data, labels

## Case 1: Univariate Data Likelihood

In [ ]:
mean1 = [-2]
mean2 = [1]
var1 = [1]
var2 = [2]
N1 = 50 # C1 - blue
N2 = 100 # C2 - red

data, labels = generateData(mean1, mean2, var1, var2, N1, N2)

In [ ]:
def drawMAP(data, labels):
    
    #### Estimate parameters (MLE solution)
    # Means
    mu1 = np.mean(data[labels==1]) 
    mu2 = np.mean(data[labels==2])
    
    # Variances
    var1 = np.cov(data[labels==1])
    var2 = np.cov(data[labels==2])
    
    #### Estimate Prior Probabilities - relative frequency
    N = len(data)
    N1 = np.sum(labels==1)
    N2 = N - N1
    p1 = N1/N # prior probability for C1
    p2 = N2/N # prior probability for C2
    
    #### Define data likelihoods 
    G1=stats.norm(loc=mu1,scale=np.sqrt(var1)) # P(x|C1)
    G2=stats.norm(loc=mu2,scale=np.sqrt(var2)) # P(x|C2)
    x=np.linspace(-6,6,1001)
        
    #### Plot the weighted densities
    # these are proportional to the posteriors
    plt.figure(figsize=(10,5))
    plt.plot(x,p1*G1.pdf(x),label='$f_X(x|C_1)P(C_1)$')
    plt.plot(x,p2*G2.pdf(x),label='$f_X(x|C_2)P(C_2)$')
    
    #### Determine the regions where the posterior for deciding C1 
    # and the posterior for deciding C2
    R1=x[np.where(p1*G1.pdf(x)>= p2*G2.pdf(x))]
    R2=x[np.where(p1*G1.pdf(x)< p2*G2.pdf(x))]

    # Fill under the regions found above
    plt.fill_between(R1,p1*G1.pdf(R1),alpha=0.3,label='Decide C1')
    plt.fill_between(R2,p2*G2.pdf(R2),alpha=0.3,label='Decide C2')
    plt.scatter(data[labels==1], -0.01*np.ones(N1), c='b', alpha=0.5, edgecolors='k')
    plt.scatter(data[labels==2], -0.01*np.ones(N2), c='r', alpha=0.5, edgecolors='k')
    plt.legend()
    
    # Print the MAP threshold
    print('MAP decision threshold to decide C2 is >',round(R2[0],2))

In [ ]:
drawMAP(data,labels)

## Case 2: Multivariate Data Likelihood

In [ ]:
mean1 = [-1, -1]
mean2 = [1, 1]
cov1 = [[1,0],[0,1]]
cov2 = [[1,0],[0,1]]
N1 = 50
N2 = 100

data, labels = generateData(mean1, mean2, cov1, cov2, N1, N2)

In [ ]:
data.shape

In [ ]:
plt.scatter(data[labels==1,0], data[labels==1,1], color='red', label='C1')
plt.scatter(data[labels==2,0], data[labels==2,1], color='blue', label='C2')
plt.legend();

In [ ]:
#### Estimate parameters (with MLE solutions)
# Means
mu1 = np.mean(data[labels==1], axis=0)
print('Mean of Class 1: ', mu1)
mu2 = np.mean(data[labels==2], axis=0)
print('Mean of Class 2: ', mu2)

# Covariances - in this example we are showing the case where we estimate the full covariance
cov1 = np.cov(data[labels==1,:].T) # np.cov expects input to be D-by-N
print('Covariance of Class 1: ',cov1)
cov2 = np.cov(data[labels==2, :].T)
print('Covariance of Class 2: ',cov2)

#### Estimate Prior Probabilities
N = N1+N2
p1 = N1/N
print('Probability of  Class 1: ',p1)
p2 = N2/N
print('Probability of  Class 2: ',p2)

In [ ]:
# Compute a grid of values for x and y 
grid = 4
x = np.linspace(-grid, grid, 100)
y = np.linspace(-grid, grid, 100)
xm, ym = np.meshgrid(x, y)
X = np.flip(np.dstack([xm,ym]),axis=0) # grid of values

# Let's plot the probabaility density function (pdf) for each class
y1 = stats.multivariate_normal.pdf(X, mean=mu1, cov=cov1) #P(x|C1) - data likelihood for C1
y2 = stats.multivariate_normal.pdf(X, mean=mu2, cov=cov2) #P(x|C2)

fig =plt.figure(figsize=(15,5))
fig.add_subplot(1,2,1)
plt.imshow(y1, extent=[-grid,grid,-grid,grid])
plt.colorbar()
plt.xlabel('Feature 1'); plt.ylabel('Feature 2')
plt.title('PDF Likelihood for Class 1')

fig.add_subplot(1,2,2)
plt.imshow(y2, extent=[-grid,grid,-grid,grid])
plt.colorbar()
plt.xlabel('Feature 1'); plt.ylabel('Feature 2')
plt.title('PDF Likelihood for Class 2');

In [ ]:
fig =plt.figure(figsize=(15,5))
fig.add_subplot(1,2,1)
plt.scatter(data[labels==1,0], data[labels==1,1], c='r',alpha=0.3)
plt.imshow(y1, extent=[-grid,grid,-grid,grid],cmap='YlOrRd')
plt.colorbar()
plt.xlabel('Feature 1'); plt.ylabel('Feature 2')
plt.title('PDF Likelihood for Class 1')

fig.add_subplot(1,2,2)
plt.scatter(data[labels==2,0], data[labels==2,1], c='b',alpha=0.3)
plt.imshow(y2, extent=[-grid,grid,-grid,grid], cmap='Blues')
plt.colorbar()
plt.xlabel('Feature 1'); plt.ylabel('Feature 2')
plt.title('PDF Likelihood for Class 2');

In [ ]:
# Let's take a look at the posterior distributions: they represent our classification decision
pos1 = (y1*p1)/(y1*p1+y2*p2) # P(C1|x) - posterior probability
pos2 = (y2*p2)/(y1*p1+y2*p2) # P(C2|x)

fig =plt.figure(figsize=(15,5))
fig.add_subplot(1,2,1)
plt.imshow(pos1, extent=[-grid,grid,-grid,grid])
plt.colorbar()
plt.xlabel('Feature 1'); plt.ylabel('Feature 2')
plt.title('Posterior for Class 1')

fig.add_subplot(1,2,2)
plt.imshow(pos2, extent=[-grid,grid,-grid,grid])
plt.colorbar()
plt.xlabel('Feature 1'); plt.ylabel('Feature 2')
plt.title('Posterior for Class 2');

In [ ]:
pos1.shape

In [ ]:
# Look at the decision boundary for deciding Class 2

plt.figure(figsize=(8,5))
plt.imshow(pos2>pos1, extent=[-grid,grid,-grid,grid])
plt.colorbar()
plt.xlabel('Feature 1'); plt.ylabel('Feature 2')
plt.title('Region to Decide Class 2');

* Let's use this classifier to predict the class label for point $[1,1]$:

In [ ]:
x = [1,1]

# Data Likelihoods
y1_newPoint = stats.multivariate_normal.pdf(x, mean=mu1, cov=cov1) #P(x|C1)
y2_newPoint = stats.multivariate_normal.pdf(x, mean=mu2, cov=cov2) #P(x|C2)

print('Data likelihoods:')
print('P(x|C1) = ', y1_newPoint)
print('P(x|C2) = ', y2_newPoint,'\n')

# Posterior Probabilities
y1_pos = (y1_newPoint*p1)/(y1_newPoint*p1+y2_newPoint*p2) #P(C1|x)
y2_pos = (y2_newPoint*p2)/(y1_newPoint*p1+y2_newPoint*p2) #P(C2|x)

print('Posterior probabilities:')
print('P(C1|x) = ', y1_pos)
print('P(C2|x) = ', y2_pos,'\n')

if y1_pos > y2_pos:
    print('x = ',x,' belongs to class 1')
else:
    print('x = ',x,' belongs to class 2')

* What about $x=[4,4]$?

In [ ]:
x = [4,4]

# Data Likelihoods
y1_newPoint = stats.multivariate_normal.pdf(x, mean=mu1, cov=cov1) #P(x|C1)
y2_newPoint = stats.multivariate_normal.pdf(x, mean=mu2, cov=cov2) #P(x|C2)

print('Data likelihoods:')
print('P(x|C1) = ', y1_newPoint)
print('P(x|C2) = ', y2_newPoint,'\n')

# Posterior Probabilities
y1_pos = (y1_newPoint*p1)/(y1_newPoint*p1+y2_newPoint*p2) #P(C1|x)
y2_pos = (y2_newPoint*p2)/(y1_newPoint*p1+y2_newPoint*p2) #P(C2|x)

print('Posterior probabilities:')
print('P(C1|x) = ', y1_pos)
print('P(C2|x) = ', y2_pos,'\n')

if y1_pos > y2_pos:
    print('x = ',x,' belongs to class 1')
else:
    print('x = ',x,' belongs to class 2')

---

# Discriminant Functions

Classification can also be seen as implementing a set of **discriminant functions**, $g_i(x), i=1,\dots, K$, such that we

$$\text{Choose} \;\; C_i \;\; \text{if} \;\; g_i(x) = \max_k g_k(x)$$

where $g_i(\mathbf{x}) = \ln(P(C_i|\mathbf{x})) = \ln(P(\mathbf{x}|C_i)P(C_i))$.

When there are **two classes** ($C_1$ and $C_2$), we have the **Bayesian decision rule**

\begin{align*}
\text{Choose} \;\; C_1 \;\; \text{if} \;\; & P(C_1|x) > P(C_2|x) \\
\text{Choose} \;\; C_1 \;\; \text{if} \;\; & P(x|C_1)P(C_1) > P(x|C_2)P(C_2)\\
\text{Choose} \;\; C_1 \;\; \text{if} \;\; & \ln(P(x|C_1)P(C_1)) > \ln(P(x|C_2)P(C_2))\\
\text{Choose} \;\; C_1 \;\; \text{if} \;\; & g_1(x) > g_2(x)\\
\text{Choose} \;\; C_1 \;\; \text{if} \;\; & g_1(x)-g_2(x) > 0
\end{align*}


When there are **two classes**, we can define a single discriminant

$$g(\mathbf{x}) = g_1(\mathbf{x}) - g_2(\mathbf{x})$$

and we

$$\text{Choose} \begin{cases}C_1 & \text{if} \; g(\mathbf{x})>0\\ C_2 & \text{otherwise}\end{cases}$$

---

## Exercise 1

Explicitly calculate the decision boundary for the two-class two-dimensional data. Assume that the data likelihood for each class is a bivariate Gaussian distribution

$$P(\mathbf{x}|C_i) = \frac{1}{(2\pi)^{d/2}|\Sigma_i|^{1/2}} \exp\left\{-\frac{1}{2}(\mathbf{x}-\mu_i)^T\Sigma_i^{-1}(\mathbf{x}-\mu_i)\right\}$$

where 

$$\mu_1 =\begin{bmatrix}3\\6\end{bmatrix}, \;\;\; \mu_2 =\begin{bmatrix}3\\-2\end{bmatrix}, \;\;\; \Sigma_1=\begin{bmatrix}1/2 & 0\\0 &2\end{bmatrix}, \;\;\; \Sigma_2=\begin{bmatrix}2 & 0\\0 &2\end{bmatrix}$$

The inverse matrices are

$$\Sigma_1^{-1}=\begin{bmatrix}2 & 0\\0 &1/2\end{bmatrix}, \;\;\; \Sigma_2^{-1}=\begin{bmatrix}1/2 & 0\\0 &1/2\end{bmatrix}$$

Assume equal prior probabilities $P(C_1)=P(C_2)=\frac{1}{2}$.

1. Compute the discriminant function (decision function).

**Answer in board notes.**

In [ ]:
mu1 = [3, 6]
mu2 = [3, -2]

Sigma1 = np.array([[0.5,0],[0,2]])
Sigma2 = np.array([[2,0],[0,2]])

p1 = 0.5
p2 = 1-p1

In [ ]:
# Compute a grid of values for x and y 
gridx = 15
gridy = 20
x = np.linspace(-gridx, gridx, 500)
y = np.linspace(-10, gridy, 500)
xm, ym = np.meshgrid(x, y)
X = np.flip(np.dstack([xm,ym]),axis=0) # grid of values

# Let's plot the probabaility density function (pdf) for each class
y1 = stats.multivariate_normal.pdf(X, mean=mu1, cov=Sigma1) #P(x|C1) - data likelihood for C1
y2 = stats.multivariate_normal.pdf(X, mean=mu2, cov=Sigma2) #P(x|C2)

fig =plt.figure(figsize=(15,5))
fig.add_subplot(1,2,1)
plt.imshow(y1, extent=[-gridx,gridx,-10,gridy])
plt.colorbar()
plt.xlabel('Feature 1'); plt.ylabel('Feature 2')
plt.title('PDF Likelihood for Class 1')

fig.add_subplot(1,2,2)
plt.imshow(y2, extent=[-gridx,gridx,-10,gridy])
plt.colorbar()
plt.xlabel('Feature 1'); plt.ylabel('Feature 2')
plt.title('PDF Likelihood for Class 2');

In [ ]:
# Let's take a look at the posterior distributions: they represent our classification decision
pos1 = (y1*p1)/(y1*p1 + y2*p2) # P(C1|x) - posterior probability
pos2 = (y2*p2)/(y1*p1 + y2*p2) # P(C2|x)

# Look at the decision boundary:
plt.figure(figsize=(8,5))
plt.imshow(pos1>pos2, extent=[-gridx,gridx,-10,gridy])
plt.colorbar()
plt.xlabel('Feature 1'); plt.ylabel('Feature 2')
plt.title('Region to Decide Class 1');

In [ ]:
x1 = np.linspace(-6.8,12.8, 100)

plt.figure(figsize=(8,5))
plt.imshow(pos1>pos2, extent=[-gridx,gridx,-10,gridy])
plt.plot(x1, 3.514 - 1.125*x1 + 0.1875*x1**2, 'r', label='Discriminant function')
plt.colorbar(); plt.legend()
plt.xlabel('Feature 1'); plt.ylabel('Feature 2')
plt.title('Region to Decide Class 1');

---
---
---

# What if the data likelihood looks like this?

In [ ]:
from sklearn.datasets import make_blobs

X, t = make_blobs(n_samples=200, centers=5, n_features=2)
plt.scatter(X[:,0], X[:,1],color='blue', alpha=0.5, edgecolor='k')
plt.xlabel('Feature $x_1$',size=15); plt.ylabel('Feature $x_2$',size=15);
plt.title('Data likelihood $f_X([x_1,x_2])$',size=15);

If we assume a single Gaussian distribution, we would obtain a very poor estimate of the true underlying data likelihood.

We can better represent this data with a **mixture model**:

$$p(x|\Theta) = \sum_{k=1}^K \pi_k P(x|\Theta_k)$$

where $\Theta = \{\Theta_k\}_{k=1}^K$ are set of parameters that define the distributional form in the probabilistic model $P(\bullet|\Theta_k)$ and 

\begin{align*}
0 & \leq \pi_k \leq 1\\
& \sum_k \pi_k = 1
\end{align*}